In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import scipy.constants as const
import autograd.numpy as anp
import pandas as pd

from autograd import grad

$$
H = \sum_{i=1}^{N} \frac{1}{2} m_i |\mathbf{v}_i|^2 - \sum_{i=1}^{N} \sum_{j=i+1}^{N} \frac{G m_i m_j}{r_{ij}}
$$

In [3]:

def T(state_vectors):
    masses = anp.array([body.m for body in state_vectors], dtype=float)
    velocities = anp.array([body.v for body in state_vectors], dtype=float)
    T_total = anp.sum(0.5 * masses * anp.linalg.norm(velocities, axis=1) ** 2)
    return T_total

def V(state_vectors):
    masses = anp.array([body.m for body in state_vectors], dtype=float)
    positions = anp.array([body.r for body in state_vectors], dtype=float)
    V_total = 0.0
    num_bodies = len(state_vectors)
    for i in range(num_bodies):
        for j in range(i + 1, num_bodies):
            r_ij = anp.linalg.norm(positions[i] - positions[j])
            V_total -= masses[i] * masses[j] / r_ij
    return V_total
    
def H(state_vectors):

    H_total = T(state_vectors) + V(state_vectors)
    
    return H_total

$$
\dot{\mathbf{r}}_{i, 1}^{\text{Ham}} = \frac{\partial H}{\partial \mathbf{p}_{i, 1}}= \frac{1}{m_i}  \frac{\partial H}{\partial\mathbf{v}_{i, 1}^{\text{pred}}}
$$

In [4]:
def H_velocity_gradient(state_vectors):
    def H_velocities(velocities_flat):
        velocities = velocities_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(body.r[0], body.r[1], body.r[2],
                                          velocities[i, 0], velocities[i, 1], velocities[i, 2],
                                          body.a[0], body.a[1], body.a[2], body.m, body.t) 
                              for i, body in enumerate(state_vectors)]
        return H(temp_state_vectors)

    velocities_flat = anp.array([body.v for body in state_vectors], dtype=float).flatten()
    grad_H = grad(H_velocities)
    grad_values = grad_H(velocities_flat).reshape(len(state_vectors), 3)
    masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = np.round(grad_values / masses, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

hamiltonian = H([body_one, body_two, body_three])
print("Hamiltonian:", hamiltonian)

gradient = H_velocity_gradient([body_one, body_two, body_three])
print("Gradient of Hamiltonian w.r.t velocity:")
print(gradient)

Hamiltonian: 9996.292893218813
Gradient of Hamiltonian w.r.t velocity:
        Gradient X  Gradient Y  Gradient Z
Body 0       100.0         0.0         0.0
Body 1         0.0         1.0         0.0
Body 2         0.0         0.0         1.0


$$
  \dot{\mathbf{v}}_{i, 1}^{\text{Ham}} = \frac{\dot{\mathbf{p}}_{i, 1}}{m_i} = \frac{-1}{m_i} \frac{\partial H}{\partial \mathbf{r}_{i, 1}^{\text{pred}
  }}
$$

In [5]:
def H_position_gradient(state_vectors):
    def H_positions(positions_flat):
        positions = positions_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(positions[i, 0], positions[i, 1], positions[i, 2],
                                          body.v[0], body.v[1], body.v[2],
                                          body.a[0], body.a[1], body.a[2], body.m, body.t) 
                              for i, body in enumerate(state_vectors)]
        return H(temp_state_vectors)

    positions_flat = anp.array([body.r for body in state_vectors], dtype=float).flatten()
    grad_H = grad(H_positions)
    grad_values = grad_H(positions_flat).reshape(len(state_vectors), 3)
    masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = -anp.round(grad_values / masses, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

gradient = H_position_gradient([body_one, body_two, body_three])
print("Gradient of Hamiltonian w.r.t position:")
print(gradient)

Gradient of Hamiltonian w.r.t position:
        Gradient X  Gradient Y  Gradient Z
Body 0    1.000000    1.000000        -0.0
Body 1   -2.353553    0.353553        -0.0
Body 2    0.353553   -2.353553        -0.0


$$
   \mathbf{p}_{i,\frac{1}{2}}^{\text{leapfrog}} = \mathbf{p}_{i,0}^{\text{true}} - \frac{\Delta t}{2} \frac{\partial V}{\partial \mathbf{r}_{i,0}^{\text{true}}}.
$$
where $$\mathbf{p}_{i,0}^{\text{true}} = m_i\mathbf{v}_{i,0}^{\text{true}}$$

$$
\mathbf{r}_{i,1}^{\text{leapfrog}} = \mathbf{r}_{i,0}^{\text{true}} + \Delta t \, \frac{\mathbf{p}_{i,\frac{1}{2}}^{\text{leapfrog}}}{m_i}.
$$

$$
\mathbf{p}_{i,1}^{\text{leapfrog}} = \mathbf{p}_{i,\frac{1}{2}}^{\text{leapfrog}} - \frac{\Delta t}{2} \frac{\partial V}{\partial \mathbf{r}_{i,1}^{\text{leapfrog}}}.
$$

In [6]:
def V_position_gradient(state_vectors):
    def V_positions(positions_flat):
        positions = positions_flat.reshape(len(state_vectors), 3)
        temp_state_vectors = [StateVector(positions[i, 0], positions[i, 1], positions[i, 2],
                                          body.v[0], body.v[1], body.v[2],
                                          body.a[0], body.a[1], body.a[2], body.m, body.t) 
                              for i, body in enumerate(state_vectors)]
        return V(temp_state_vectors)

    positions_flat = anp.array([body.r for body in state_vectors], dtype=float).flatten()
    grad_V = grad(V_positions)
    grad_values = grad_V(positions_flat).reshape(len(state_vectors), 3)
    # masses = anp.array([body.m for body in state_vectors], dtype=float).reshape(-1, 1)
    result = -anp.round(grad_values, 10).astype(float)

    df = pd.DataFrame(result, columns=['Gradient X', 'Gradient Y', 'Gradient Z'], index = [f'Body {i}' for i in range(len(state_vectors))])
    
    return df

gradient = V_position_gradient([body_one, body_two, body_three])
print("Gradient of Potential Energy w.r.t position:")
print(gradient)

Gradient of Potential Energy w.r.t position:
        Gradient X  Gradient Y  Gradient Z
Body 0    2.000000    2.000000        -0.0
Body 1   -2.353553    0.353553        -0.0
Body 2    0.353553   -2.353553        -0.0


In [7]:
def leapfrog_update(state_vectors, dt):
    p_0_true = anp.array([body.m * body.v for body in state_vectors], dtype=float)
    r_0_true = anp.array([body.r for body in state_vectors], dtype=float)
    mass_vector = anp.array([body.m for body in state_vectors], dtype=float)

    # Compute gradient of potential energy w.r.t. position (force)
    gradient_V_0 = V_position_gradient(state_vectors).values
    p_leapfrog_one_half = p_0_true - (0.5 * dt * gradient_V_0)

    r_leapfrog_one = r_0_true + dt * (p_leapfrog_one_half / mass_vector[:, None])
    for i, body in enumerate(state_vectors):
        body.r = r_leapfrog_one[i]
        
    
    gradient_V_1 = V_position_gradient(state_vectors).values
    p_leapfrog_one = p_leapfrog_one_half - (0.5 * dt * gradient_V_1)
    for i, body in enumerate(state_vectors):
        body.r = r_leapfrog_one[i]
        body.v = p_leapfrog_one[i] / body.m
        body.s = anp.array([body.r, body.v, gradient_V_1[i]], dtype=float)  # Store latest state

    return state_vectors


body_one = StateVector(0, 0, 0, 100, 0, 0, 0, 0, 0, 2, 0)
body_two = StateVector(1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0)
body_three = StateVector(0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0)

print("Initial state vectors:")
for body in [body_one, body_two, body_three]:
    print(body.r)

state_vector_updated = leapfrog_update([body_one, body_two, body_three], 0.05)

print("Updated state vectors:")
for body in state_vector_updated:
    print(body.r)

Initial state vectors:
[0. 0. 0.]
[1. 0. 0.]
[0. 1. 0.]
Updated state vectors:
[ 4.99875e+00 -1.25000e-03  0.00000e+00]
[1.00294194 0.04955806 0.        ]
[-4.41941738e-04  1.00294194e+00  5.00000000e-02]
